In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


c:\Users\alsin\miniconda3\envs\intro-ai\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


<h1> Import Data </h1>

In [2]:
df = pd.read_csv("../datafiles/boston.csv")

Input features in order:
1. CRIM: Per capita crime rate by town
2. ZN: Proportion of residential land zones for lots over 25k sq.ft
3. INDUS: Proportion of non-retail business acress per town
4. CHAS: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5. NOX: Nitric oxide contentration (parts per 10 million)
6. RM: Average number of rooms per dwelling
7. AGE: proportion of owner-occupied unites built prior to 1940.
8. DIS: Weighted distances to five Boston employment centres
9. RAD: Index of accessibility to radial highways
10. TAX: Full-value property-tax rate per 10.000 NOK
11. PTRATIO: Pupil-teacher ratio by town
12. B: The result of the equation B  = 100(Bk-0.63)^2 where Bk is the proportion of the black demografic by town
13. LSTAT: Percentage of lower status of the population


Output variable:
1. MEDV: Median value of owner-occupied homes in 1000's [k]

In [3]:
# As always it is a good idea to get familiar with the data, check the shape, the first 5 rows, df.describe, df.info():
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     534 non-null    float64
 1   ZN       533 non-null    float64
 2   INDUS    534 non-null    float64
 3   CHAS     534 non-null    int64  
 4   NOX      532 non-null    float64
 5   RM       531 non-null    float64
 6   AGE      533 non-null    float64
 7   DIS      531 non-null    float64
 8   RAD      531 non-null    float64
 9   TAX      533 non-null    float64
 10  PTRATIO  533 non-null    float64
 11  B        531 non-null    float64
 12  LSTAT    533 non-null    float64
 13  MEDV     533 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 58.5 KB


<h1>Data cleaning </h1>
Remove NaN values

In [5]:
df = df.dropna()

In [6]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000
mean,3.675341,10.759690,11.178566,0.065891,0.556189,6.275950,69.267636,3.755469,9.724806,409.765504,18.502132,358.108547,12.843043,719.417636
std,8.504473,22.480313,6.783069,0.248333,0.115660,0.707891,27.914387,2.077423,8.790211,169.729257,2.142337,89.057923,7.178291,11211.442227
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,-27.100000
25%,0.083827,0.000000,5.190000,0.000000,0.449000,5.887750,45.550000,2.084875,4.000000,278.500000,17.400000,375.997500,7.197500,16.475000
50%,0.260420,0.000000,9.690000,0.000000,0.538000,6.221000,77.750000,3.190950,5.000000,330.000000,19.100000,391.705000,11.655000,21.050000
75%,3.840055,12.500000,18.100000,0.000000,0.631000,6.604500,94.300000,5.104475,24.000000,666.000000,20.200000,396.280000,17.152500,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,190000.500000


<h2>Handle outliers </h2>

In [7]:
# Let us find some outliers. MEDV looks like a good place to begin:
df[df["MEDV"] < 0]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,-27.1
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,-18.9
151,1.62864,0.0,21.89,0,0.624,5.019,100.0,1.4394,4.0,437.0,21.2,396.90,34.41,-14.4
330,0.31827,0.0,9.90,0,0.544,5.914,83.2,3.9986,4.0,304.0,18.4,390.70,18.33,-17.8
461,14.42080,0.0,18.10,0,0.740,6.461,93.3,2.0026,24.0,666.0,20.2,27.49,18.05,-9.6
489,5.82115,0.0,18.10,0,0.713,6.513,89.9,2.8016,24.0,666.0,20.2,393.82,10.29,-20.2
495,13.07510,0.0,18.10,0,0.580,5.713,56.7,2.8237,24.0,666.0,20.2,396.90,14.76,-20.1


In [8]:
df[df["MEDV"] > 30000]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
487,3.69311,0.0,18.1,0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,170000.7
488,6.65492,0.0,18.1,0,0.713,6.317,83.0,2.7344,24.0,666.0,20.2,396.90,13.99,190000.5


Let us remove these, but also drop the complete duplicates from our dataframe:

In [9]:
df = df[df["MEDV"] > 0]
df = df[df["MEDV"] < 30000]
df = df.drop_duplicates()

In [10]:
# Let us now inspect our data again:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 533
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     480 non-null    float64
 1   ZN       480 non-null    float64
 2   INDUS    480 non-null    float64
 3   CHAS     480 non-null    int64  
 4   NOX      480 non-null    float64
 5   RM       480 non-null    float64
 6   AGE      480 non-null    float64
 7   DIS      480 non-null    float64
 8   RAD      480 non-null    float64
 9   TAX      480 non-null    float64
 10  PTRATIO  480 non-null    float64
 11  B        480 non-null    float64
 12  LSTAT    480 non-null    float64
 13  MEDV     480 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 56.2 KB


<h1>Model creation </h1>
Now we will use XGBoost to create the regression model. XGBoost: Extreme gradient boosting. It is a very popular algorithm used in machine learning and in the industry.
I will use it for regression models, but it can also be used for classification.

In [11]:
model = xgb.XGBRegressor()

In [12]:
# Y is the target column, and x is the rest of the df.
X = df.drop("MEDV", axis=1)
y = df["MEDV"]

In [13]:
# We will now use train_test_split to split the data set into training and test data. If we were to have a cleaned data set, with both test and train data this would not be needed. 
# If we were to train the model on the whole dataset, then it woould learn the dataset perfectly, but we will not know how it performs on unseen data.

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)

In [14]:
# Use the training set (X_train, y_train) to train the model by calling the .fit() method:
model.fit(X_train, y_train)

c:\Users\alsin\miniconda3\envs\intro-ai\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [15]:
# Use the model to predict the target values for the test set (X_test)
predictions = model.predict(X_test)
print(predictions)

[24.383587  15.6437025 11.81969   24.0624     7.2313447 23.396069
 31.875507   9.9234    20.74392   14.157346  21.419746  23.288025
 13.006834  20.199545  15.244674  26.165443  29.739359  10.754356
 21.694517  20.300098  40.152378  15.50556   21.701298  22.341156
 18.451683  20.162655  21.132065  20.251835  22.523315  29.215408
 14.913843  20.383339  12.599338  14.901775  10.61583   21.657026
 23.78104   34.782078  20.777437  13.074869  25.842125  31.60973
 23.671621  34.546158  29.749992  19.46539   19.61928   21.867052
 31.823517  27.82488   34.657764  14.55689   10.531318  14.800061
 21.426765  31.467226  22.495838  20.210964  25.708975  21.820425
 17.692743  44.424667  26.510231  15.019946  20.491055  19.572826
 19.094353  23.916357  25.308989  40.391262  24.261845  24.895916
 25.922731  21.417156  28.023756  27.119045  47.385284  22.549677
 23.245436  24.08684   15.393285  19.851433  28.652666  15.399486
 17.909578  28.054457  22.194563  22.254976  32.55484   20.884851
 16.399323 

In [16]:
# Now we will find the mean squaret error and root mean squared error for the predictions. Lower mse is better. We will finds rmse and mse between predictions and y_test
mse = mean_squared_error(predictions, y_test)
rmse = np.sqrt(mse)

# Printing the mse to see how much, on average, our model is off (squared). Also the RMSE
print(mse)
print(rmse)

10.68097416865667
3.268175969659019


<h1> Improving our model with Hyperparameter tuning </h1>

In [17]:
# Taken from lab solution, these are some of the hypermparameters you can tune for XGBoost:
# A hyperparameter is a parameter that is not learned by the model, but is set by the user.
# The parameters that are learned by the model are called model parameters.
# The model starts off with some default values for the hyperparameters, but you can change them to get potentially better results.
# This process is called hyperparameter tuning.

# If you want, you can adjust the hyperparameters and see if you get a better result

params = {
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7],
    "n_estimators": [100, 200, 300, 400, 500, 900, 1100, 1500]
}

In [18]:
# Use RandomizedSearchCV to find the best hyperparameters for the model. Random search is a method for hyperparameter tuning that will try a given number of random combinations of hyperparameters.
# Use the training set to instantiate the random search by calling the .fit() method with the test set.
# n_iter is the number of iterations to run the random search. Let us try a fairly small number, so we don't have to sit here all day

# First, we create a new, simliar model, but with default hyperparameters. We do not fit this model with the training set.add
model2 = xgb.XGBRegressor()

random_search = RandomizedSearchCV(model2, param_distributions=params, n_iter=50, scoring="neg_mean_squared_error", n_jobs=-1, cv=5)

# Fit the model with x and y train sets
random_search.fit(X_train, y_train)

c:\Users\alsin\miniconda3\envs\intro-ai\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=...
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 900, 1100,
                                                         1500]},
                   scoring='neg_mean_squared_error')

In [19]:
model_new = random_search.best_estimator_

In [20]:
type(model_new)

xgboost.sklearn.XGBRegressor

In [21]:
# Now we create new predictions with the new model
predictions = model_new.predict(X_test)

In [22]:
mse_new = mean_squared_error(predictions, y_test)

mse_new

11.258830634610343

In [23]:
# It is even higher, possibly because of the low number of n_iters:
print(f"Relation between better new and old error on the models: {(mse_new/mse)}")

Relation between better new and old error on the models: 1.0541014758419127
